In [4]:
import os
from params import PARAMS
import pickle
import numpy as np
import json
import pandas as pd
from main import show_metrics

In [11]:
final_results = pd.read_csv('output/lr-0.0001/results.csv')
final_results.groupby(['model_name']).mean()

,random_seed,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params
model_name,,,,,,,,
imagine_cnn,688933.0,0.385417,0.386774,0.388957,0.387482,0.385417,0.591561,371220.0
imagine_fcn,688933.0,0.338542,0.337983,0.339785,0.339104,0.338542,0.559393,354308.0
imagine_gcn,688933.0,0.310764,0.312613,0.313616,0.313674,0.310764,0.542167,354308.0
imagine_gcn_auto,688933.0,0.309028,0.306174,0.309001,0.306265,0.309028,0.537832,358404.0
imagine_gcram_auto,688933.0,0.340278,0.341480,0.343985,0.340992,0.340278,0.560751,1434668.0
imagine_rnn,688933.0,0.276042,0.276219,0.276735,0.277561,0.276042,0.518121,357556.0


In [ ]:
final_results = pd.read_csv('output/2021-12-10-01-58/results.csv')
final_results.groupby(['model_name']).mean()

In [2]:


std_per_dataset = final_results.groupby(['model_name', 'dataset_name']).std().drop(columns=['random_seed'])
std_per_dataset = std_per_dataset.rename(columns={'accuracy': 'accuracy_std',
                    'precision_macro': 'precision_macro_std', 
                    'precision_weighted': 'precision_weighted_std', 
                    'recall_macro': 'recall_macro_std', 
                    'recall_weighted': 'recall_weighted_std'})
std_per_dataset = std_per_dataset.drop(columns=['AUROC', 'n_params'])
mean_per_dataset = final_results.groupby(['model_name', 'dataset_name']).mean().drop(columns='random_seed')
results_per_dataset = pd.concat([mean_per_dataset, std_per_dataset], axis=1)
results_per_dataset.reset_index()

,model_name,dataset_name,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params,accuracy_std,precision_macro_std,precision_weighted_std,recall_macro_std,recall_weighted_std
0,imagine_cnn,cross_subject_data_0_5_subjects,0.384375,0.386071,0.387294,0.385825,0.384375,0.590353,371220.0,0.024231,0.023899,0.024515,0.025071,0.024231
1,imagine_cnn,cross_subject_data_1_5_subjects,0.323958,0.323835,0.324766,0.324660,0.323958,0.549626,371220.0,0.007009,0.006349,0.006862,0.008325,0.007009
2,imagine_cnn,cross_subject_data_2_5_subjects,0.352778,0.352863,0.355403,0.352573,0.352778,0.568384,371220.0,0.025260,0.026813,0.028377,0.025493,0.025260
3,imagine_cnn,cross_subject_data_3_5_subjects,0.301042,0.301397,0.302674,0.301692,0.301042,0.534382,371220.0,0.019356,0.020264,0.020588,0.019836,0.019356
4,imagine_cnn,cross_subject_data_4_5_subjects,0.310764,0.311063,0.311579,0.311573,0.310764,0.540877,371220.0,0.015431,0.015960,0.015858,0.015914,0.015431
5,imagine_fcn,cross_subject_data_0_5_subjects,0.343403,0.344230,0.346135,0.344457,0.343403,0.562927,354308.0,0.018994,0.019789,0.020327,0.021110,0.018994
6,imagine_fcn,cross_subject_data_1_5_subjects,0.313194,0.315142,0.316961,0.313343,0.313194,0.542265,354308.0,0.023441,0.027294,0.026989,0.024943,0.023441
7,imagine_fcn,cross_subject_data_2_5_subjects,0.316667,0.320231,0.321895,0.319024,0.316667,0.545736,354308.0,0.024966,0.025759,0.027748,0.025504,0.024966
8,imagine_fcn,cross_subject_data_3_5_subjects,0.277778,0.277775,0.278723,0.278269,0.277778,0.518704,354308.0,0.033597,0.034352,0.033429,0.034905,0.033597
9,imagine_fcn,cross_subject_data_4_5_subjects,0.286111,0.284851,0.285361,0.286039,0.286111,0.523891,354308.0,0.022881,0.024564,0.023719,0.023116,0.022881


In [3]:
std_per_model = final_results.groupby(['model_name']).std().drop(columns=['random_seed'])
std_per_model = std_per_model.rename(columns={'accuracy': 'accuracy_std',
                    'precision_macro': 'precision_macro_std', 
                    'precision_weighted': 'precision_weighted_std', 
                    'recall_macro': 'recall_macro_std', 
                    'recall_weighted': 'recall_weighted_std'})
std_per_model = std_per_model.drop(columns=['AUROC', 'n_params'])
mean_per_model = final_results.groupby(['model_name']).mean().drop(columns='random_seed')
results_per_model = pd.concat([mean_per_model, std_per_model], axis=1)
results_per_model

,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params,accuracy_std,precision_macro_std,precision_weighted_std,recall_macro_std,recall_weighted_std
model_name,,,,,,,,,,,,
imagine_cnn,0.334583,0.335046,0.336343,0.335265,0.334583,0.556725,371220.0,0.035722,0.036374,0.036921,0.036091,0.035722
imagine_fcn,0.307431,0.308446,0.309815,0.308226,0.307431,0.538705,354308.0,0.033213,0.034846,0.035220,0.034223,0.033213
imagine_gcn,0.268681,0.269229,0.270859,0.268888,0.268681,0.512610,354308.0,0.017946,0.017994,0.017983,0.017804,0.017946
imagine_gcn_auto,0.310764,0.310546,0.311935,0.311470,0.310764,0.540887,358404.0,0.028052,0.028809,0.028780,0.028876,0.028052
imagine_rnn,0.256181,0.255668,0.256924,0.256358,0.256181,0.504184,357556.0,0.020622,0.020304,0.020807,0.020328,0.020622


In [ ]:
path = 'output/imagine_cnn/cross_subject_data_0/195645/results.pickle'
results = pickle.load(open(path, 'rb'))

In [ ]:
results['history']['val_acc']

[0.28912869095802307,
 0.2906447947025299,
 0.30393293499946594,
 0.29135823249816895,
 0.28877195715904236,
 0.29626327753067017,
 0.2916257977485657,
 0.294568806886673,
 0.2949255406856537,
 0.30009809136390686,
 0.2949255406856537,
 0.29474717378616333,
 0.2933202385902405,
 0.30214929580688477,
 0.29082316160202026,
 0.28029966354370117,
 0.29296353459358215,
 0.2868099510669708,
 0.29465797543525696,
 0.29421207308769226,
 0.2916257977485657,
 0.2868991196155548,
 0.2931418716907501,
 0.29055559635162354,
 0.2914474308490753,
 0.28377774357795715,
 0.2889503240585327,
 0.29055559635162354,
 0.28841522336006165,
 0.29251760244369507,
 0.29180416464805603,
 0.2860964834690094,
 0.28663158416748047,
 0.2899313271045685,
 0.2874342203140259,
 0.2857397496700287,
 0.29171496629714966,
 0.28288593888282776,
 0.29091233015060425,
 0.28636404871940613,
 0.2914474308490753,
 0.2894854247570038,
 0.2901988625526428,
 0.2857397496700287,
 0.2838669419288635,
 0.28618568181991577,
 0.2915365